## Merging df_real + df_anon

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import copy
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()
client = Client(cluster)
client

### reading real data

In [4]:
# Chargement sous forme de dask dataframe. Adresse du fichier à changer.
dd_traces = dd.read_csv("../INSAnonym/dataset.csv", sep = '\t', header = None, \
                        names = ['ID', 'DateTime', 'lat', 'lon']).rename(columns = {'lat':'lon', 'lon':'lat'})
# Fenêtres de 15min : fait sur le dask dataframe (parallélisation)
T_agreg = 15 # Temps en secondes
freq_str = f'{15}min'
truncated_datetime_col = f'DateTime{freq_str}'

dd_traces[truncated_datetime_col] = dd_traces.DateTime.astype(np.datetime64).dt.floor("15T").astype(str)


dd_traces['week_number'] = dd_traces.DateTime.astype(np.datetime64).dt.isocalendar().week
# Identifiant : id-week_number
dd_traces['id_week'] = dd_traces.ID.astype(str) + '-' + dd_traces.week_number.astype(str)

dd_traces

,ID,DateTime,lon,lat,DateTime15min,week_number,id_week
npartitions=28,,,,,,,
,int64,object,float64,float64,object,UInt32,object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


### reading anonymized data (after clustering + shuffling + geo-indistinguishability)

In [3]:
import pickle

with open('../data/anon_ag_15_nbc_5_nbt_4_eps_0.0001.pkl', 'rb') as f:
    df_anon = pickle.load(f)

    
df_anon.reset_index(inplace=True, drop=True)
df_anon['DateTime15min'] = df_anon['DateTime15min'].astype(str)
df_anon    

,ID,DateTime15min,Date,lat,lon,clustered_lat,clustered_lon,Delete_cluster,time,day,shuff_lat,shuff_lon,radius_of_gyration,geoinds_lat,geoindis_lon
0,1,2015-03-04 00:00:00,2015-03-04,45.772198,4.870504,45.771152,4.892673,False,NIGHT,2015-03-04,45.772198,4.870504,2592.454493,45.425912,4.632550
1,2,2015-03-04 00:00:00,2015-03-04,45.786265,4.879282,45.771152,4.892673,False,NIGHT,2015-03-04,45.770001,4.869676,108.231059,45.295942,5.194751
2,4,2015-03-04 00:00:00,2015-03-04,45.755808,4.866166,45.771152,4.892673,False,NIGHT,2015-03-04,45.770207,4.870024,581.929523,45.913148,4.606333
3,6,2015-03-04 00:00:00,2015-03-04,45.632137,5.146270,45.771152,4.892673,False,NIGHT,2015-03-04,45.770285,4.869538,115.060888,45.323527,4.947874
4,7,2015-03-04 00:00:00,2015-03-04,45.786242,4.879744,45.771152,4.892673,False,NIGHT,2015-03-04,45.770036,4.869665,30.113446,46.129348,4.478864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146865,73,2015-05-12 23:15:00,2015-05-12,45.738155,4.852832,45.747083,4.859102,True,NIGHT,2015-05-12,48.891537,2.343147,180.921024,45.557337,4.648068
146866,72,2015-05-12 23:30:00,2015-05-12,45.756012,4.865372,45.747083,4.859102,True,NIGHT,2015-05-12,48.891537,2.343147,187.863223,45.586662,5.084817
146867,73,2015-05-12 23:30:00,2015-05-12,45.738155,4.852832,45.747083,4.859102,True,NIGHT,2015-05-12,48.891536,2.343146,180.921024,45.268921,4.835539
146868,72,2015-05-12 23:45:00,2015-05-12,45.756012,4.865372,45.747083,4.859102,True,NIGHT,2015-05-12,48.891543,2.343156,187.863223,45.585950,4.888863


In [4]:
df_anon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146870 entries, 0 to 146869
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ID                  146870 non-null  int64  
 1   DateTime15min       146870 non-null  object 
 2   Date                146870 non-null  object 
 3   lat                 146870 non-null  float64
 4   lon                 146870 non-null  float64
 5   clustered_lat       146870 non-null  float64
 6   clustered_lon       146870 non-null  float64
 7   Delete_cluster      146870 non-null  bool   
 8   time                146870 non-null  object 
 9   day                 146870 non-null  object 
 10  shuff_lat           146870 non-null  float64
 11  shuff_lon           146870 non-null  float64
 12  radius_of_gyration  146870 non-null  float64
 13  geoinds_lat         146870 non-null  float64
 14  geoindis_lon        146870 non-null  float64
dtypes: bool(1), float64(9), int64(1), 

## create dictionary with indexed tuple (id + timestamp 15 min)

In [6]:
df_anon.drop(['lat','lon','day','shuff_lat','shuff_lon'], axis=1, inplace=True)
df_anon.set_index(["ID","DateTime15min"], inplace=True)
dico_anon = df_anon.to_dict(orient="index")

dd_traces["Delete_cluster"] = dd_traces.apply(lambda x: int(dico_anon[tuple((x.ID, x.DateTime15min))]["Delete_cluster"]) if tuple((x.ID, x.DateTime15min)) in dico_anon.keys() else np.nan, axis=1, meta='float64')

#dd_traces["geoinds_lat"] = dd_traces.apply(lambda x: dico_anon[tuple((x.ID, x.DateTime15min))]["geoinds_lat"] if tuple((x.ID, x.DateTime15min)) in dico_anon.keys() else np.nan, axis=1 , meta='float64')
#dd_traces["geoindis_lon"] = dd_traces.apply(lambda x: dico_anon[tuple((x.ID, x.DateTime15min))]["geoindis_lon"] if tuple((x.ID, x.DateTime15min)) in dico_anon.keys() else np.nan, axis=1 , meta='float64')
dd_traces["geoinds_lat"] = dd_traces.apply(lambda x: dico_anon[tuple((x.ID, x.DateTime15min))]["geoinds_lat"] if tuple((x.ID, x.DateTime15min)) in dico_anon.keys() else 0.0, axis=1 , meta='float64')
dd_traces["geoindis_lon"] = dd_traces.apply(lambda x: dico_anon[tuple((x.ID, x.DateTime15min))]["geoindis_lon"] if tuple((x.ID, x.DateTime15min)) in dico_anon.keys() else 0.0, axis=1 , meta='float64')
dd_traces["time"] = dd_traces.apply(lambda x: dico_anon[tuple((x.ID, x.DateTime15min))]["time"] if tuple((x.ID, x.DateTime15min)) in dico_anon.keys() else np.nan, axis=1 , meta='float64')

## Transform from dask to pandas dataframe

In [7]:
df_real = dd_traces.compute() # C'est ce qui est  en temps
df_real.reset_index(inplace=True, drop=True)
df_real

,ID,DateTime,lon,lat,DateTime15min,week_number,id_week,Delete_cluster,geoinds_lat,geoindis_lon,time
0,1,2015-03-04 00:35:16,4.870147,45.772140,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT
1,1,2015-03-04 00:35:48,4.870218,45.772095,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT
2,1,2015-03-04 00:35:49,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT
3,1,2015-03-04 00:35:50,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT
4,1,2015-03-04 00:35:52,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT
...,...,...,...,...,...,...,...,...,...,...,...
34551844,110,2015-03-12 16:23:21,2.343094,48.891650,2015-03-12 16:15:00,11,110-11,NaN,0.000000,0.000000,NaN
34551845,110,2015-03-12 16:23:22,2.343094,48.891650,2015-03-12 16:15:00,11,110-11,NaN,0.000000,0.000000,NaN
34551846,110,2015-03-12 16:23:24,2.343094,48.891649,2015-03-12 16:15:00,11,110-11,NaN,0.000000,0.000000,NaN
34551847,110,2015-03-12 16:23:25,2.343094,48.891649,2015-03-12 16:15:00,11,110-11,NaN,0.000000,0.000000,NaN


## Add pseudonymization and save backup df

In [8]:
"""
# Création des pseudonyms par id-week : assez long
id_week_unique = df_real['id_week'].drop_duplicates()
df_pseudonyms = pd.DataFrame({'id_week':id_week_unique, 'pseudo':pd.util.hash_pandas_object(id_week_unique)})
df_test = df_real.merge(df_pseudonyms, on = 'id_week', how = 'left').rename(columns = {'ID':'ID_true', 'pseudo':'ID'})
df_test = df_test.astype({'ID':str})
"""

import hashlib

lst_id_week = np.unique(df_real['id_week'])
lst_hash_id_week = set()

for val in lst_id_week:
    hash_value = int(hashlib.sha1(val.encode("utf-8")).hexdigest(), 16) % (10 ** 8)
    if hash_value not in lst_hash_id_week:
        lst_hash_id_week.add(hash_value)
    else:
        while True:
            val = val+val
            hash_value = int(hashlib.sha1(val.encode("utf-8")).hexdigest(), 16) % (10 ** 8)
            if hash_value not in lst_hash_id_week:
                lst_hash_id_week.add(hash_value)
                break
dic_hash = dict(zip(lst_id_week,lst_hash_id_week))

df_real['pseudo'] = df_real['id_week'].map(dic_hash)
df_real.rename(columns = {'ID':'ID_true', 'pseudo':'ID'}, inplace=True)
df_real


,ID_true,DateTime,lon,lat,DateTime15min,week_number,id_week,Delete_cluster,geoinds_lat,geoindis_lon,time,ID
0,1,2015-03-04 00:35:16,4.870147,45.772140,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904
1,1,2015-03-04 00:35:48,4.870218,45.772095,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904
2,1,2015-03-04 00:35:49,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904
3,1,2015-03-04 00:35:50,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904
4,1,2015-03-04 00:35:52,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904
...,...,...,...,...,...,...,...,...,...,...,...,...
34551844,110,2015-03-12 16:23:21,2.343094,48.891650,2015-03-12 16:15:00,11,110-11,NaN,0.000000,0.000000,NaN,10029129
34551845,110,2015-03-12 16:23:22,2.343094,48.891650,2015-03-12 16:15:00,11,110-11,NaN,0.000000,0.000000,NaN,10029129
34551846,110,2015-03-12 16:23:24,2.343094,48.891649,2015-03-12 16:15:00,11,110-11,NaN,0.000000,0.000000,NaN,10029129
34551847,110,2015-03-12 16:23:25,2.343094,48.891649,2015-03-12 16:15:00,11,110-11,NaN,0.000000,0.000000,NaN,10029129


In [9]:
df_test=df_real.copy()


In [10]:
df_test.to_csv("../data/df_anon_complete_pseudo.csv", index=False)

## Check if dataset is ok

In [11]:
df_test = pd.read_csv("../data/df_anon_complete_pseudo.csv")
df_test = df_test.astype({'ID':str})
df_test.fillna('DEL', inplace=True)
df_test['DateTime'] = pd.to_datetime(df_test['DateTime'], format='%Y-%m-%d %H:%M:%S')
df_test['day'] = df_test['DateTime'].dt.date.astype(str)

In [12]:
df_test



,ID_true,DateTime,lon,lat,DateTime15min,week_number,id_week,Delete_cluster,geoinds_lat,geoindis_lon,time,ID,day
0,1,2015-03-04 00:35:16,4.870147,45.772140,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904,2015-03-04
1,1,2015-03-04 00:35:48,4.870218,45.772095,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904,2015-03-04
2,1,2015-03-04 00:35:49,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904,2015-03-04
3,1,2015-03-04 00:35:50,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904,2015-03-04
4,1,2015-03-04 00:35:52,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904,2015-03-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34551844,110,2015-03-12 16:23:21,2.343094,48.891650,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,10029129,2015-03-12
34551845,110,2015-03-12 16:23:22,2.343094,48.891650,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,10029129,2015-03-12
34551846,110,2015-03-12 16:23:24,2.343094,48.891649,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,10029129,2015-03-12
34551847,110,2015-03-12 16:23:25,2.343094,48.891649,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,10029129,2015-03-12


# Applying Sampling strategy: according to median of nb reports per user/time

## Dic for median per day/time

In [13]:
df_nb_rep_day_time_id = df_test.groupby(by=['day','time','ID']).count()
df_nb_rep_day_time_id

ID_true  DateTime    lon    lat  DateTime15min  \
day        time  ID                                                         
2015-03-04 DEL   11712901     2331      2331   2331   2331           2331   
                 15875053     1622      1622   1622   1622           1622   
                 18309748      652       652    652    652            652   
                 20847543    16609     16609  16609  16609          16609   
                 22582614       63        63     63     63             63   
...                            ...       ...    ...    ...            ...   
2015-05-12 NIGHT 46519844      146       146    146    146            146   
                 75656872     2669      2669   2669   2669           2669   
                 77796995     2387      2387   2387   2387           2387   
                 79358688     1715      1715   1715   1715           1715   
                 98497810     2386      2386   2386   2386           2386   

                           week_number  id_week  Delete_cluster  geoinds_lat  \
day        time  ID                                                            
2015-03-04 DEL   11712901         2331     2331            2331         2331   
                 15875053         1622     1622            1622         1622   
                 18309748          652      652             652          652   
                 20847543        16609    16609           16609        16609   
                 22582614           63       63              63           63   
...                                ...      ...             ...          ...   
2015-05-12 NIGHT 46519844          146      146             146          146   
                 75656872         2669     2669            2669         2669   
                 77796995         2387     2387            2387         2387   
                 79358688         1715     1715            1715         1715   
                 98497810         2386     2386            2386         2386   

                           geoindis_lon  
day        time  ID                      
2015-03-04 DEL   11712901          2331  
                 15875053          1622  
                 18309748           652  
                 20847543         16609  
                 22582614            63  
...                                 ...  
2015-05-12 NIGHT 46519844           146  
                 75656872          2669  
                 77796995          2387  
                 79358688          1715  
                 98497810          2386  

[4535 rows x 10 columns]

In [14]:
lst_days = np.unique(df_test['day'])
lst_time =  np.unique(df_test['time'])
lst_time = lst_time[lst_time !='DEL']

dic_median = {day:{time:0 for time in lst_time} for day in lst_days}

for day in lst_days:
    for time in lst_time:
        try:
            dic_median[day][time] = df_nb_rep_day_time_id.loc[day, time]['lon'].median()
        except:
            pass

## Dic smp rate per ID/day/time

In [15]:
df_nb_rep_id_day_time = df_test.groupby(by=['ID','day','time']).count()#['DateTime']
df_nb_rep_id_day_time

ID_true  DateTime    lon    lat  DateTime15min  \
ID       day        time                                                      
10029129 2015-03-09 DEL         1322      1322   1322   1322           1322   
         2015-03-10 DEL         2635      2635   2635   2635           2635   
                    WORK        1956      1956   1956   1956           1956   
         2015-03-11 DEL         1340      1340   1340   1340           1340   
         2015-03-12 DEL          889       889    889    889            889   
...                              ...       ...    ...    ...            ...   
99901002 2015-03-04 NIGHT       4430      4430   4430   4430           4430   
         2015-03-05 NIGHT      12732     12732  12732  12732          12732   
         2015-03-06 DEL         1967      1967   1967   1967           1967   
         2015-03-07 DEL         7337      7337   7337   7337           7337   
                    WEEKEND     3058      3058   3058   3058           3058   

                             week_number  id_week  Delete_cluster  \
ID       day        time                                            
10029129 2015-03-09 DEL             1322     1322            1322   
         2015-03-10 DEL             2635     2635            2635   
                    WORK            1956     1956            1956   
         2015-03-11 DEL             1340     1340            1340   
         2015-03-12 DEL              889      889             889   
...                                  ...      ...             ...   
99901002 2015-03-04 NIGHT           4430     4430            4430   
         2015-03-05 NIGHT          12732    12732           12732   
         2015-03-06 DEL             1967     1967            1967   
         2015-03-07 DEL             7337     7337            7337   
                    WEEKEND         3058     3058            3058   

                             geoinds_lat  geoindis_lon  
ID       day        time                                
10029129 2015-03-09 DEL             1322          1322  
         2015-03-10 DEL             2635          2635  
                    WORK            1956          1956  
         2015-03-11 DEL             1340          1340  
         2015-03-12 DEL              889           889  
...                                  ...           ...  
99901002 2015-03-04 NIGHT           4430          4430  
         2015-03-05 NIGHT          12732         12732  
         2015-03-06 DEL             1967          1967  
         2015-03-07 DEL             7337          7337  
                    WEEKEND         3058          3058  

[4535 rows x 10 columns]

In [16]:
smp_min = 0.9 # uniform subsample rate

lst_days = np.unique(df_test['day'])
lst_ids = np.unique(df_test['ID'])
lst_time =  np.unique(df_test['time'])

dic_smp_rate = {user_id:{day:{time:0 for time in lst_time} for day in lst_days} for user_id in lst_ids}

for user_id in lst_ids:
    for day in lst_days:
        for time in lst_time:
            if time != 'DEL':
                try:
                    nb_rep = df_nb_rep_id_day_time.loc[user_id, day, time]['lon']

                    median_day_time = int(smp_min * dic_median[day][time])

                    dic_smp_rate[user_id][day][time] = smp_min if nb_rep <= median_day_time else median_day_time/nb_rep
                except:
                    pass
            else:
                dic_smp_rate[user_id][day][time] = 1 #we don't care if its already deleted ID
#dic_smp_rate

## Apply 'Delete' (format of submission) for rows with nan values

In [17]:
import time
starttime = time.time()

df_test['selected'] = df_test.apply(lambda x: int(np.random.random() <= dic_smp_rate[x.ID][x.day][x.time]), axis=1)

df_test["ID"] = df_test.apply(lambda x: "DEL" if x.selected == 0 else x.ID, axis=1)
df_test["ID"] = df_test.apply(lambda x: "DEL" if x.Delete_cluster  else x.ID, axis=1)
df_test["ID"] = df_test.apply(lambda x: "DEL" if x.geoinds_lat == 'DEL' else x.ID, axis=1)
df_test.to_csv("../data/df_anon_complete_del.csv", index=False)
print(time.time() - starttime)
df_test

1986.8665556907654


,ID_true,DateTime,lon,lat,DateTime15min,week_number,id_week,Delete_cluster,geoinds_lat,geoindis_lon,time,ID,day,selected
0,1,2015-03-04 00:35:16,4.870147,45.772140,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,DEL,2015-03-04,0
1,1,2015-03-04 00:35:48,4.870218,45.772095,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,DEL,2015-03-04,0
2,1,2015-03-04 00:35:49,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904,2015-03-04,1
3,1,2015-03-04 00:35:50,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,DEL,2015-03-04,0
4,1,2015-03-04 00:35:52,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904,2015-03-04,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34551844,110,2015-03-12 16:23:21,2.343094,48.891650,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,DEL,2015-03-12,1
34551845,110,2015-03-12 16:23:22,2.343094,48.891650,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,DEL,2015-03-12,1
34551846,110,2015-03-12 16:23:24,2.343094,48.891649,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,DEL,2015-03-12,1
34551847,110,2015-03-12 16:23:25,2.343094,48.891649,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,DEL,2015-03-12,1


## Checking final dataset

In [18]:
df_final = pd.read_csv("../data/df_anon_complete_del.csv")
df_final

,ID_true,DateTime,lon,lat,DateTime15min,week_number,id_week,Delete_cluster,geoinds_lat,geoindis_lon,time,ID,day,selected
0,1,2015-03-04 00:35:16,4.870147,45.772140,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,DEL,2015-03-04,0
1,1,2015-03-04 00:35:48,4.870218,45.772095,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,DEL,2015-03-04,0
2,1,2015-03-04 00:35:49,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904,2015-03-04,1
3,1,2015-03-04 00:35:50,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,DEL,2015-03-04,0
4,1,2015-03-04 00:35:52,4.870210,45.772072,2015-03-04 00:30:00,10,1-10,0.0,45.540461,4.758914,NIGHT,40144904,2015-03-04,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34551844,110,2015-03-12 16:23:21,2.343094,48.891650,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,DEL,2015-03-12,1
34551845,110,2015-03-12 16:23:22,2.343094,48.891650,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,DEL,2015-03-12,1
34551846,110,2015-03-12 16:23:24,2.343094,48.891649,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,DEL,2015-03-12,1
34551847,110,2015-03-12 16:23:25,2.343094,48.891649,2015-03-12 16:15:00,11,110-11,DEL,0.000000,0.000000,DEL,DEL,2015-03-12,1


In [19]:
df_test = pd.read_csv("../data/df_anon_complete_pseudo.csv")
lst_ids = list(np.unique(df_test['ID']))
lst_ids = [str(val) for val in lst_ids] + ['DEL']
df_test.groupby('ID').count()

,ID_true,DateTime,lon,lat,DateTime15min,week_number,id_week,Delete_cluster,geoinds_lat,geoindis_lon,time
ID,,,,,,,,,,,
260105,32587,32587,32587,32587,32587,32587,32587,21132,32587,32587,21132
481457,149269,149269,149269,149269,149269,149269,149269,68352,149269,149269,68352
485052,35739,35739,35739,35739,35739,35739,35739,31960,35739,35739,31960
485666,187755,187755,187755,187755,187755,187755,187755,107286,187755,187755,107286
687675,51215,51215,51215,51215,51215,51215,51215,38605,51215,51215,38605
...,...,...,...,...,...,...,...,...,...,...,...
99286544,198794,198794,198794,198794,198794,198794,198794,116058,198794,198794,116058
99311145,183814,183814,183814,183814,183814,183814,183814,112250,183814,183814,112250
99780803,16059,16059,16059,16059,16059,16059,16059,9705,16059,16059,9705


In [20]:
df_final.groupby('ID').count()

,ID_true,DateTime,lon,lat,DateTime15min,week_number,id_week,Delete_cluster,geoinds_lat,geoindis_lon,time,day,selected
ID,,,,,,,,,,,,,
10029129,2047,2047,2047,2047,2047,2047,2047,2047,2047,2047,2047,2047,2047
10064322,11921,11921,11921,11921,11921,11921,11921,11921,11921,11921,11921,11921,11921
10224149,8740,8740,8740,8740,8740,8740,8740,8740,8740,8740,8740,8740,8740
10558985,3036,3036,3036,3036,3036,3036,3036,3036,3036,3036,3036,3036,3036
10875216,16715,16715,16715,16715,16715,16715,16715,16715,16715,16715,16715,16715,16715
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9933520,43425,43425,43425,43425,43425,43425,43425,43425,43425,43425,43425,43425,43425
99780803,5481,5481,5481,5481,5481,5481,5481,5481,5481,5481,5481,5481,5481
99805625,14056,14056,14056,14056,14056,14056,14056,14056,14056,14056,14056,14056,14056


#Conclusion: df_final was subsampled in comparison with df_test, and IDs were correctly hashed

## Save final dataset with no header and in 'original format', i.e., inverted lat / lon (?)

In [21]:
col_keep = ['ID', 'DateTime15min', 'geoindis_lon', 'geoinds_lat']

df_final[col_keep].to_csv('../data/df_final.csv', index=False, header=False,sep="\t")

In [22]:
import pandas as pd
#df_final.to_pickle('../data/df_final.pkl')
dff = pd.read_csv('../data/df_final.csv', header=None)
dff.to_pickle('../data/df_final.pkl')

